In [31]:
#in this notebook we will implement MOCO paper:
#https://arxiv.org/pdf/1911.05722.pdf
#CNN architecture we change, but MOCO loss function is the same, and training is the same
#we use 5 different data gmentations (rotations, blur, color distortion, cropping and resizing) for defining the positive samples

#training is done on 1 GPU, training setting are:
#1. Use the entire training data to learn the representations.
#2. Once the representations are learned, use a linear and logistic layers and retrain with 10-50% of supervised training data.
#3. Experiment with two different sizes for the encoder dictionary.

In [32]:
version = 1
experiment = 'moco'
experiment_name = experiment + '_' + str(version)

In [33]:
#imports
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torchsummary import summary

from functools import partial
from torch.utils.data import DataLoader
from torchvision import transforms
from tqdm import tqdm
import numpy as np

In [34]:
from PIL import Image
#tensorboard
from torch.utils.tensorboard import SummaryWriter


In [35]:
import importlib

import models
importlib.reload(models)
from models import  pentaClassifier, binaryClassifier, baseClassifier, convNet


import config
from config import cfg
importlib.reload(config)



<module 'config' from '/home/lisa/bhartendu/adrl/A3/config.py'>

In [36]:
#device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

#### CIFAR10

In [37]:
from torchvision.datasets import CIFAR10

In [38]:
import argparse
import json
import math
import os
import pandas as pd

In [39]:
#seed
seed = 42
os.environ['PYTHONHASHSEED'] = str(seed)
# Torch RNG
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)



#### HYPER - PARAMETERS

In [40]:
results_dir = 'results/'
moco_pretrained_dir = results_dir+'model_'+experiment_name

In [59]:
moco_pretrained_dir

'results/model_moco_1'

In [41]:
lr = 0.06
epochs = 2500  # 200
batch_size = 512
schedule = [120, 160]
wd = 5e-4
cos = True      # cosine lr schedule

In [42]:
moco_dim = 512
moco_k = 4096
moco_m = 0.99
moco_t = 0.07
moco_loss_symmetric = True

In [43]:
bn_splits = 8

## data-loader

#### Adapted from pytorch code of Contrastive learning libs: http://github.com/zhirongw/lemniscate.pytorch

In [44]:
class CIFAR10Pair(CIFAR10):
    """CIFAR10 Dataset.
    """
    def __getitem__(self, index):
        img = self.data[index]
        img = Image.fromarray(img)

        if self.transform is not None:
            im_1 = self.transform(img)
            im_2 = self.transform(img)

        return im_1, im_2

# train_transform = transforms.Compose([
#     transforms.RandomResizedCrop(32),
#     transforms.RandomHorizontalFlip(p=0.5),
#     transforms.RandomApply([transforms.ColorJitter(0.4, 0.4, 0.4, 0.1)], p=0.8),
#     transforms.RandomGrayscale(p=0.2),
#     transforms.ToTensor(),
#     transforms.Normalize([0.4914, 0.4822, 0.4465], [0.2023, 0.1994, 0.2010])])
#we will aplly 5 different data gmentations (rotations, blur, color distortion, cropping and resizing) for defining the positive samples
train_transform = transforms.Compose([
    transforms.RandomResizedCrop(32),
    #random rotation with random angle
    transforms.RandomRotation([-1* torch.rand(1)*360 , torch.rand(1)*360]),
    #random blur
    transforms.RandomApply([transforms.GaussianBlur(3, sigma=[0.1, 2.0])], p=0.5),
    #random color distortion
    transforms.RandomApply([transforms.ColorJitter(0.4, 0.4, 0.4, 0.1)], p=0.8),
    transforms.ToTensor(),
    transforms.Normalize([0.4914, 0.4822, 0.4465], [0.2023, 0.1994, 0.2010])])
    

test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.4914, 0.4822, 0.4465], [0.2023, 0.1994, 0.2010])])

# data prepare
train_data = CIFAR10Pair(root='data', train=True, transform=train_transform, download=True)
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True, num_workers=16, pin_memory=True, drop_last=True)

memory_data = CIFAR10(root='data', train=True, transform=test_transform, download=True)
memory_loader = DataLoader(memory_data, batch_size=batch_size, shuffle=False, num_workers=16, pin_memory=True)

test_data = CIFAR10(root='data', train=False, transform=test_transform, download=True)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False, num_workers=16, pin_memory=True)


Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


## Split Batch implementation adapted from: 
#### https://github.com/davidcpage/cifar10-fast/blob/master/torch_backend.py

In [45]:
class SplitBatchNorm(nn.BatchNorm2d):
    def __init__(self, num_features, num_splits= bn_splits, **kw):
        super().__init__(num_features, **kw)
        self.num_splits = num_splits
        
    def forward(self, input):
        N, C, H, W = input.shape
        if self.training or not self.track_running_stats:
            running_mean_split = self.running_mean.repeat(self.num_splits)
            running_var_split = self.running_var.repeat(self.num_splits)
            outcome = nn.functional.batch_norm(
                input.view(-1, C * self.num_splits, H, W), running_mean_split, running_var_split, 
                self.weight.repeat(self.num_splits), self.bias.repeat(self.num_splits),
                True, self.momentum, self.eps).view(N, C, H, W)
            self.running_mean.data.copy_(running_mean_split.view(self.num_splits, C).mean(dim=0))
            self.running_var.data.copy_(running_var_split.view(self.num_splits, C).mean(dim=0))
            return outcome
        else:
            return nn.functional.batch_norm(
                input, self.running_mean, self.running_var, 
                self.weight, self.bias, False, self.momentum, self.eps)

#### the following class was adapted from facebookresearch/moco which replaces 

In [46]:
# #write a function to replace all the batchnorm layers with splitbatchnorm layers if bn_splits > 1
# def replace_bn(model, num_splits):
#     #if num_splits is 1, then return the model
#     if num_splits == 1:
#         return model
#     norm_layer = partial(SplitBatchNorm, num_splits=bn_splits) if bn_splits > 1 else nn.BatchNorm2d
#     for name, child in model.named_children():
#         if isinstance(child, nn.BatchNorm2d):
#             setattr(model, name, norm_layer(child.num_features))
#         else:
#             replace_bn(child, num_splits)
#     return model
    

            

In [47]:
#function to use the class SplitBatchNorm to replace the batchnorm layers with splitbatchnorm layers if bn_splits > 1
#the init has num_features and num_splits as arguments, thus while changing the batchnorm layers to splitbatchnorm layers, we need to pass the num_splits as an argument
def replace_bn(model, num_splits):
    #if num_splits is 1, then return the model
    if num_splits == 1:
        return model
    #loop through the model and replace the batchnorm layers with splitbatchnorm layers using the class SplitBatchNorm
    norm_layer = partial(SplitBatchNorm, num_splits=bn_splits)
    for name, child in model.named_children():
        if isinstance(child, nn.BatchNorm2d):
            #change the batchnorm layers to splitbatchnorm layers by the return of the function : SplitBatchNorm(child.num_features, num_splits)
            #replace batchnorm with norm_layer
            setattr(model, name, norm_layer(child.num_features))
        else:
            replace_bn(child, num_splits)

## Model

In [48]:
#create a model class named ModelBAse that will have the exact same architecture as the convNet class but will flatten the output of the model, to get embedding of the model
class ModelBase(nn.Module):
    def __init__(self, cfg=cfg['model'], device=device):
        super().__init__()
        self.cfg = cfg
        self.device = device
        self.conv = convNet(cfg, device=device)
        self.flat = nn.Flatten()
        self.fc = nn.Linear(moco_dim, moco_dim)
        
        

        
    def forward(self, x):
        x = self.flat(self.conv(x))
        #pass the output of the model through a linear layer
        x = self.fc(x)

        return x


In [49]:
#buils an object of the ModelBase class
model = ModelBase()


-----------------------------------------------------------
ConvNet
ModuleList(
  (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
  (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU()
  (3): Dropout(p=0.3, inplace=False)
  (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (5): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), bias=False)
  (6): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (7): ReLU()
  (8): Dropout(p=0.3, inplace=False)
  (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (10): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), bias=False)
  (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (12): ReLU()
  (13): Dropout(p=0.3, inplace=False)
  (14): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
)
---------------------------------------------

In [50]:
#summary of the model
summary(model, input_size=(3, 32, 32), device='cpu')

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 30, 30]             864
       BatchNorm2d-2           [-1, 32, 30, 30]              64
              ReLU-3           [-1, 32, 30, 30]               0
           Dropout-4           [-1, 32, 30, 30]               0
         MaxPool2d-5           [-1, 32, 15, 15]               0
            Conv2d-6           [-1, 64, 13, 13]          18,432
       BatchNorm2d-7           [-1, 64, 13, 13]             128
              ReLU-8           [-1, 64, 13, 13]               0
           Dropout-9           [-1, 64, 13, 13]               0
        MaxPool2d-10             [-1, 64, 6, 6]               0
           Conv2d-11            [-1, 128, 4, 4]          73,728
      BatchNorm2d-12            [-1, 128, 4, 4]             256
             ReLU-13            [-1, 128, 4, 4]               0
          Dropout-14            [-1, 12

---------------------------------------------------------------------------
---------------------------------------------------------------------------

## MOCO

#### defininng MOCO Training Routine: Including the queue and the memory bank and key encoder, query encoder




In [51]:
class ModelMoCo(nn.Module):
    def __init__(self, dim=moco_dim, K=moco_k, m=moco_m, T=moco_t, bn_splits=bn_splits, symmetric=True):
        super(ModelMoCo, self).__init__()

        self.K = K
        self.m = m
        self.T = T
        self.symmetric = symmetric

        # create the encoders
        self.encoder_q = ModelBase()
        self.encoder_k = ModelBase()

        for param_q, param_k in zip(self.encoder_q.parameters(), self.encoder_k.parameters()):
            param_k.data.copy_(param_q.data)  # initialize
            param_k.requires_grad = False  # not update by gradient

        # create the queue
        self.register_buffer("queue", torch.randn(dim, K))
        self.queue = nn.functional.normalize(self.queue, dim=0)

        self.register_buffer("queue_ptr", torch.zeros(1, dtype=torch.long))

    @torch.no_grad()
    def _momentum_update_key_encoder(self):
        """
        Momentum update of the key encoder
        """
        for param_q, param_k in zip(self.encoder_q.parameters(), self.encoder_k.parameters()):
            param_k.data = param_k.data * self.m + param_q.data * (1. - self.m)

    @torch.no_grad()
    def _dequeue_and_enqueue(self, keys):
        batch_size = keys.shape[0]

        ptr = int(self.queue_ptr)
        assert self.K % batch_size == 0  # for simplicity

        # replace the keys at ptr (dequeue and enqueue)
        self.queue[:, ptr:ptr + batch_size] = keys.t()  # transpose
        ptr = (ptr + batch_size) % self.K  # move pointer

        self.queue_ptr[0] = ptr

    @torch.no_grad()
    def _batch_shuffle_single_gpu(self, x):
        """
        Batch shuffle, for making use of BatchNorm.
        """
        # random shuffle index
        idx_shuffle = torch.randperm(x.shape[0]).cuda()

        # index for restoring
        idx_unshuffle = torch.argsort(idx_shuffle)

        return x[idx_shuffle], idx_unshuffle

    @torch.no_grad()
    def _batch_unshuffle_single_gpu(self, x, idx_unshuffle):
        """
        Undo batch shuffle.
        """
        return x[idx_unshuffle]

    def contrastive_loss(self, im_q, im_k):
        # compute query features
        q = self.encoder_q(im_q)  # queries: NxC
        q = nn.functional.normalize(q, dim=1)  # already normalized

        # compute key features
        with torch.no_grad():  # no gradient to keys
            # shuffle for making use of BN
            im_k_, idx_unshuffle = self._batch_shuffle_single_gpu(im_k)

            k = self.encoder_k(im_k_)  # keys: NxC
            k = nn.functional.normalize(k, dim=1)  # already normalized

            # undo shuffle
            k = self._batch_unshuffle_single_gpu(k, idx_unshuffle)

        # compute logits
        # Einstein sum is more intuitive
        # positive logits: Nx1
        l_pos = torch.einsum('nc,nc->n', [q, k]).unsqueeze(-1)
        # negative logits: NxK
        l_neg = torch.einsum('nc,ck->nk', [q, self.queue.clone().detach()])

        # logits: Nx(1+K)
        logits = torch.cat([l_pos, l_neg], dim=1)

        # apply temperature
        logits /= self.T

        # labels: positive key indicators
        labels = torch.zeros(logits.shape[0], dtype=torch.long).cuda()
        
        loss = nn.CrossEntropyLoss().cuda()(logits, labels)

        return loss, q, k

    def forward(self, im1, im2):
        """
        Input:
            im_q: a batch of query images
            im_k: a batch of key images
        Output:
            loss
        """

        # update the key encoder
        with torch.no_grad():  # no gradient to keys
            self._momentum_update_key_encoder()

        # compute loss
        if self.symmetric:  # asymmetric loss
            loss_12, q1, k2 = self.contrastive_loss(im1, im2)
            loss_21, q2, k1 = self.contrastive_loss(im2, im1)
            loss = loss_12 + loss_21
            k = torch.cat([k1, k2], dim=0)
        else:  # asymmetric loss
            loss, q, k = self.contrastive_loss(im1, im2)

        self._dequeue_and_enqueue(k)

        return loss



#### Optimizer: SGD

In [52]:
# define optimizer
optimizer = torch.optim.SGD(model.parameters(), lr=lr, weight_decay=wd, momentum=0.9)

##### learning rate schedule

In [53]:
# lr scheduler for training
def adjust_learning_rate(optimizer, epoch):
    """Decay the learning rate based on schedule"""
    learning_rate = lr 
    if cos:  # cosine lr schedule
        learning_rate *= 0.5 * (1. + math.cos(math.pi * epoch / epochs))
    else:  # stepwise lr schedule
        for milestone in schedule:
            learning_rate *= 0.1 if epoch >= milestone else 1.
    for param_group in optimizer.param_groups:
        param_group['lr'] = learning_rate

#### Training Loop

In [54]:
# train for one epoch
#create batch writer
batch_writer = SummaryWriter(f'{results_dir}/run/')
batch_count = 0


def train(net, data_loader, train_optimizer, epoch):
    net.train()
    adjust_learning_rate(optimizer, epoch)

    total_loss, total_num, train_bar = 0.0, 0, tqdm(data_loader)
    for im_1, im_2 in train_bar:
        im_1, im_2 = im_1.cuda(non_blocking=True), im_2.cuda(non_blocking=True)

        loss = net(im_1, im_2)
        
        train_optimizer.zero_grad()
        loss.backward()
        train_optimizer.step()

        #write to batch writer
        #make batch count static
        global batch_count
        batch_writer.add_scalar('Loss/train_batch', loss, batch_count)
        batch_count += 1
        ########

        total_num += data_loader.batch_size
        total_loss += loss.item() * data_loader.batch_size
        train_bar.set_description('Train Epoch: [{}/{}], lr: {:.6f}, Loss: {:.4f}'.format(epoch, epochs, optimizer.param_groups[0]['lr'], total_loss / total_num))

    return total_loss / total_num



## Train

##### instantiate the model

In [55]:
# create model
model = ModelMoCo(
        dim=moco_dim,
        K=moco_k,
        m=moco_m,
        T=moco_t,

        bn_splits=bn_splits,
        symmetric=moco_loss_symmetric,
    ).to(device)
print(model.encoder_q)

-----------------------------------------------------------
ConvNet
ModuleList(
  (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
  (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU()
  (3): Dropout(p=0.3, inplace=False)
  (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (5): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), bias=False)
  (6): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (7): ReLU()
  (8): Dropout(p=0.3, inplace=False)
  (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (10): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), bias=False)
  (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (12): ReLU()
  (13): Dropout(p=0.3, inplace=False)
  (14): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
)
---------------------------------------------

In [56]:
# logging
results = {'train_loss': []}
#if results_dir not exists, then create one
if not os.path.exists(results_dir):
    os.makedirs(results_dir)
#initialize summary writer in run directory in results_dir
writer = SummaryWriter(f'{results_dir}/run/')

##### training loop

In [58]:
# training loop
epoch_start = 1
for epoch in range(epoch_start, epochs + 1):
    
    train_loss = train(model, train_loader, optimizer, epoch)
    results['train_loss'].append(train_loss)
    writer.add_scalar('Loss/train_epoch', train_loss, epoch)
    # save model if epoch is multiple of 10 or last epoch
    # if epoch % 10 == 0 :
    #     #save model
    #     # torch.save(model.state_dict(), f'{results_dir}/model'+experiment_name+'_epoch_{epoch}.pth')
    #     torch.save(model.state_dict(), moco_pretrained_dir+'_epoch_' +str(epoch) + '.pth')

    #     torch.save({'epoch': epoch, 'state_dict': model.state_dict(), 'optimizer' : optimizer.state_dict(),}, results_dir + '/model_last.pth')
    #if last epoch save model
    if epoch == epochs:
        torch.save(model.state_dict(), moco_pretrained_dir+'.pth')

Train Epoch: [1/2500], lr: 0.060000, Loss: 16.1370: 100%|██████████| 97/97 [00:07<00:00, 12.71it/s]
Train Epoch: [2/2500], lr: 0.060000, Loss: 16.1360: 100%|██████████| 97/97 [00:07<00:00, 12.65it/s]
Train Epoch: [3/2500], lr: 0.060000, Loss: 16.1220: 100%|██████████| 97/97 [00:07<00:00, 12.53it/s]
Train Epoch: [4/2500], lr: 0.060000, Loss: 16.1262: 100%|██████████| 97/97 [00:07<00:00, 12.67it/s]
Train Epoch: [5/2500], lr: 0.059999, Loss: 16.1183: 100%|██████████| 97/97 [00:07<00:00, 12.58it/s]
Train Epoch: [6/2500], lr: 0.059999, Loss: 16.1200: 100%|██████████| 97/97 [00:07<00:00, 12.71it/s]
Train Epoch: [7/2500], lr: 0.059999, Loss: 16.1071: 100%|██████████| 97/97 [00:07<00:00, 12.63it/s]
Train Epoch: [8/2500], lr: 0.059998, Loss: 16.1102: 100%|██████████| 97/97 [00:07<00:00, 12.18it/s]
Train Epoch: [9/2500], lr: 0.059998, Loss: 16.0906: 100%|██████████| 97/97 [00:07<00:00, 12.69it/s]
Train Epoch: [10/2500], lr: 0.059998, Loss: 16.1308: 100%|██████████| 97/97 [00:08<00:00, 11.96it/s]

In [ ]:
#write a function to load the trained moco model
def load_moco_model(path = moco_pretrained_dir):
    model = ModelMoCo(
        dim=moco_dim,
        K=moco_k,
        m=moco_m,
        T=moco_t,

        bn_splits=bn_splits,
        symmetric=moco_loss_symmetric,
    )
    model.load_state_dict(torch.load(path))
    return model

# Using the Learned Representations

## Test

In [ ]:
#we will test using the standard Linear classification Protocol
#the Linear classification Protocol is : as unsupervised representations are learned, use a linear and logistic layers and retrain with 10-50% of supervised training data

#we have 2 test tasks 
#TASK 1: Binary Classifier
#TASk 2: 5-class classification

In [ ]:
#we define a classifier class, that takes in number of classes
#it will first make a copy of the ModelMoCo class, load the pretrained weights, make the weights non-trainable
#then we will add a linear layer and a logistic layer to the model to match the number of classes
class Classifier(nn.Module):
    def __init__(self, num_classes):
        super(Classifier, self).__init__()
        self.moco = load_moco_model()
        #freeze the weights
        for param in self.moco.parameters():
            param.requires_grad = False
        #add a linear layer and a softmax layer
        self.linear = nn.Linear(moco_dim, num_classes)
        #add logistic layer
        self.softmax = nn.Softmax(dim=1)
    def forward(self, x):
        x = self.moco(x)
        #print the shape of x
        print("after encoder",x.shape)
        x = self.linear(x)
        #print the shape of x
        print("after linear",x.shape)
        x = self.softmax(x)
        return x


In [ ]:
#creta an object and test the model
classifier = Classifier(2)


-----------------------------------------------------------
ConvNet
ModuleList(
  (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
  (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU()
  (3): Dropout(p=0.3, inplace=False)
  (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (5): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), bias=False)
  (6): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (7): ReLU()
  (8): Dropout(p=0.3, inplace=False)
  (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (10): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), bias=False)
  (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (12): ReLU()
  (13): Dropout(p=0.3, inplace=False)
  (14): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
)
---------------------------------------------

In [ ]:
#summary

summary(model, input_size=(3, 32, 32), device='cpu')

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 30, 30]             864
       BatchNorm2d-2           [-1, 32, 30, 30]              64
              ReLU-3           [-1, 32, 30, 30]               0
           Dropout-4           [-1, 32, 30, 30]               0
         MaxPool2d-5           [-1, 32, 15, 15]               0
            Conv2d-6           [-1, 64, 13, 13]          18,432
       BatchNorm2d-7           [-1, 64, 13, 13]             128
              ReLU-8           [-1, 64, 13, 13]               0
           Dropout-9           [-1, 64, 13, 13]               0
        MaxPool2d-10             [-1, 64, 6, 6]               0
           Conv2d-11            [-1, 128, 4, 4]          73,728
      BatchNorm2d-12            [-1, 128, 4, 4]             256
             ReLU-13            [-1, 128, 4, 4]               0
          Dropout-14            [-1, 12


### TASK 1: Binary Classifier